<a href="https://colab.research.google.com/github/brendanpshea/computing_concepts_python/blob/main/sql_select_quiz/Generate_JediDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import random
from datetime import datetime, timedelta
import json

# Name components for our random name generator
name_parts = {
    'prefixes': ['Ky', 'Al', 'Ja', 'Li', 'Ka', 'Ma', 'Sa', 'Ra', 'Ta', 'Na', 'La', 'Da', 'Sha'],
    'middles': ['lo', 're', 'ri', 'na', 'ta', 'ra', 'da', 'me', 'ko', 'lu', 'ru', 'ba'],
    'suffixes': ['n', 'r', 'x', 'k', 'th', 'sx', 'kk', 'ss', 'na', 'rx', 'sk', 'nx'],
    'surnames': ['Skywalker', 'Antilles', 'Vos', 'Kenobi', 'Secura', 'Windu', 'Koon', 'Fisto',
                'Unduli', 'Billaba', 'Jinn', 'Tano', 'Ti', 'Mundi', 'Gallia']
}

planets = [
    'Tatooine', 'Coruscant', 'Naboo', 'Alderaan', 'Corellia', 'Kashyyyk', 'Ryloth',
    'Mandalore', 'Mon Cala', 'Dathomir', 'Lothal', 'Dantooine', 'Onderon', 'Tython',
    'Ilum', 'Jedha', 'Christophsis', 'Felucia', 'Geonosis', 'Mygeeto'
]

def generate_name():
    prefix = random.choice(name_parts['prefixes'])
    middle = random.choice(name_parts['middles'])
    suffix = random.choice(name_parts['suffixes'])
    surname = random.choice(name_parts['surnames'])
    return f"{prefix}{middle}{suffix} {surname}"

# Master data
masters = [
    ('Yoda', 'Council Member', 'Force Training'),
    ('Obi-Wan Kenobi', 'Master', 'Lightsaber Combat'),
    ('Mace Windu', 'Council Member', 'Vaapad Combat'),
    ('Plo Koon', 'Council Member', 'Force Judgment'),
    ('Shaak Ti', 'Council Member', 'Force Healing'),
    ('Kit Fisto', 'Master', 'Underwater Combat'),
    ('Luminara Unduli', 'Master', 'Force Push Mastery'),
    ('Quinlan Vos', 'Knight', 'Force Psychometry'),
]

# Course data
courses = [
    ('Foundations of the Force', 'Beginner', 'Basic Force sensitivity training', 1),
    ('Basic Lightsaber Training', 'Beginner', 'Fundamental lightsaber forms', 2),
    ('Advanced Combat Forms', 'Advanced', 'Advanced lightsaber techniques', 3),
    ('Meditation Techniques', 'Beginner', 'Basic meditation practices', 1),
    ('Advanced Force Control', 'Advanced', 'Mastery of Force abilities', 7),
    ('Force Healing Basics', 'Intermediate', 'Introduction to Force healing', 5),
    ('Underwater Combat Training', 'Intermediate', 'Aquatic combat techniques', 6),
    ('Stealth and Infiltration', 'Advanced', 'Force-enhanced stealth', 8),
    ('Diplomatic Relations', 'Intermediate', 'Negotiation skills', 2),
    ('Force Precognition', 'Advanced', 'Development of prescience', 1)
]

def create_database():
    conn = sqlite3.connect('jedi_academy.db')
    c = conn.cursor()

    # Enable foreign keys
    c.execute('PRAGMA foreign_keys = ON')

    # Create tables
    c.execute('''DROP TABLE IF EXISTS enrollment''')
    c.execute('''DROP TABLE IF EXISTS student''')
    c.execute('''DROP TABLE IF EXISTS course''')
    c.execute('''DROP TABLE IF EXISTS master''')
    c.execute('''DROP TABLE IF EXISTS student_profiles''')

    # Create tables with our schema
    c.execute('''CREATE TABLE master (
        master_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        rank TEXT NOT NULL,
        specialization TEXT
    )''')

    c.execute('''CREATE TABLE course (
        course_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        level TEXT NOT NULL,
        description TEXT,
        master_id INTEGER,
        FOREIGN KEY (master_id) REFERENCES master (master_id)
    )''')

    c.execute('''CREATE TABLE student (
        student_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        age INTEGER,
        home_planet TEXT,
        midichlorian_count INTEGER
    )''')

    c.execute('''CREATE TABLE enrollment (
        enrollment_id INTEGER PRIMARY KEY,
        student_id INTEGER NOT NULL,
        course_id INTEGER NOT NULL,
        start_date TEXT NOT NULL,
        grade REAL,
        FOREIGN KEY (student_id) REFERENCES student (student_id),
        FOREIGN KEY (course_id) REFERENCES course (course_id)
    )''')

    c.execute('''CREATE TABLE student_profiles (
        student_id INTEGER PRIMARY KEY,
        profile JSON
    )''')

    # Insert masters
    c.executemany('INSERT INTO master (name, rank, specialization) VALUES (?, ?, ?)',
                 masters)

    # Insert courses
    c.executemany('''INSERT INTO course (name, level, description, master_id)
                    VALUES (?, ?, ?, ?)''', courses)

    # Generate and insert students
    num_students = 200
    for _ in range(num_students):
        name = generate_name()
        age = random.randint(10, 25)
        planet = random.choice(planets)
        midichlorian = random.randint(10000, 15000)

        c.execute('''INSERT INTO student (name, age, home_planet, midichlorian_count)
                    VALUES (?, ?, ?, ?)''', (name, age, planet, midichlorian))

        student_id = c.lastrowid

        # Generate random enrollments for this student
        num_courses = random.randint(2, 6)
        course_ids = random.sample(range(1, len(courses) + 1), num_courses)
        start_date = '2024-01-15'

        enrollments = []
        for course_id in course_ids:
            grade = random.uniform(65.0, 99.0)
            c.execute('''INSERT INTO enrollment (student_id, course_id, start_date, grade)
                        VALUES (?, ?, ?, ?)''', (student_id, course_id, start_date, grade))
            enrollments.append({
                "course": courses[course_id-1][0],
                "grade": round(grade, 1),
                "completed": grade >= 70.0
            })

        # Create JSON profile
        profile = {
            "name": name,
            "age": age,
            "home_planet": planet,
            "midichlorian_count": midichlorian,
            "master": random.choice(masters),
            "skills": random.sample([
                "Force Push", "Force Pull", "Force Jump", "Telepathy",
                "Mind Trick", "Force Speed", "Force Shield", "Force Healing",
                "Lightsaber Form I", "Lightsaber Form II", "Lightsaber Form III",
                "Force Sense", "Battle Meditation", "Force Valor"
            ], random.randint(2, 5)),
            "enrollments": enrollments
        }

        c.execute('INSERT INTO student_profiles (student_id, profile) VALUES (?, ?)',
                 (student_id, json.dumps(profile)))

    conn.commit()
    conn.close()

if __name__ == '__main__':
    create_database()
    print("Database created successfully!")

    # Verify some counts
    conn = sqlite3.connect('jedi_academy.db')
    c = conn.cursor()

    c.execute('SELECT COUNT(*) FROM student')
    student_count = c.fetchone()[0]

    c.execute('SELECT COUNT(*) FROM enrollment')
    enrollment_count = c.fetchone()[0]

    c.execute('SELECT COUNT(*) FROM student_profiles')
    profile_count = c.fetchone()[0]

    print(f"Generated {student_count} students")
    print(f"Generated {enrollment_count} enrollments")
    print(f"Generated {profile_count} JSON profiles")

    conn.close()

Database created successfully!
Generated 200 students
Generated 780 enrollments
Generated 200 JSON profiles
